### 🧠 **Cross Encoder Reranker 요약**

#### 🔍 **정의**
- 질문과 문서를 함께 입력받아, **의미적으로 가장 관련성 높은 문서를 상위에 올리는** 재정렬(Reranking) 모델

#### ⚙️ **작동 방식**
- **Self-attention 기반 cross encoder** 사용
- 입력: `[질문] + [문서]`
- 출력: 문서의 **질문과의 유사도 점수**

#### ✅ **장점**
1. 더 정확한 유사도 측정
2. 의미론적 관계까지 고려 가능
3. 검색 정확도 및 RAG 성능 향상

#### ⚠️ **단점**
- 계산 비용 높음
- 대규모 데이터에는 직접 적용 어려움

#### 🛠️ **사용 방식**
- **Bi-encoder로 빠르게 후보 문서 추출 → Cross encoder로 상위 5~10개만 rerank**
- Hugging Face 모델 (`cross-encoder`, `BAAI/bge-reranker`) + LangChain으로 쉽게 통합 가능

#### ⚖️ **Trade-offs**
- 정확도 ↔ 속도
- 성능 향상 ↔ 시스템 자원 소모

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 설정
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

# 문서로부터 FAISS 인덱스 생성 및 검색기 설정
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(search_kwargs={"k": 10})

# 질의 설정
query = "Word2Vec 에 대해서 알려줄래?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

docs[0]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Document(id='9c6f94f3-7127-4047-ae90-faf2787e3f3a', metadata={'source': './data/appendix-keywords.txt'}, page_content='Open Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser')

### **Reranker 정의**

- 다국어 지원 BGE Reranker(HuggingFace): [bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3)

In [24]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# Reranker 모델 정의
model = HuggingFaceCrossEncoder(model_name = 'BAAI/bge-reranker-v2-m3')

# 상위 3개 문서 
compressor = CrossEncoderReranker(model = model, top_n = 3)

# 문서 압축 검색기 정의
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever=retriever
)

# 압축된 문서 검색 
compressed_docs = compression_retriever.invoke('Word2Vec 에 대해서 알려줄래?')

# Reranker로 인해 재정렬된 문서
compressed_docs[0].page_content

'Crawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)'